In [1]:
pip install ultralytics


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   --------------------- ------------------ 0.6/1.0 MB 17.8 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 13.2 MB/s eta 0:00:00
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.7/216.1 MB 15.3 MB/s eta 0:00:15
   ---------------------------------------- 1.7/216.1 MB 21.6 MB/s eta 0:00:10
    --------------------------------------- 2.7/216.1 MB 19.4 MB/s eta 0:00:12
    --------------------------------------- 3.6/216.1 MB 21.1 MB/s eta 0:00:11
    --------------------------------------- 4.0/216.1 MB 19.4 MB/s eta 0:00:11
    --------------------------------------- 4.0/216.1 MB 19.4 MB/s eta 0:00:11
    --------------------------------------- 4.8/216.1 MB 15.2 MB/s eta 0:00:14
  

In [1]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # Download and load the YOLOv8 nano model


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\simar\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 20.1MB/s]


In [21]:
import cv2
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import time
from sklearn.cluster import KMeans
import warnings

In [22]:
def create_water_mask(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Adjust these bounds for your canal color
    lower_water = np.array([75, 40, 60])
    upper_water = np.array([100, 200, 255])
    mask = cv2.inRange(hsv, lower_water, upper_water)
    kernel = np.ones((7,7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask

In [23]:
def create_vehicle_mask(frame, yolo_model):
    results = yolo_model(frame)
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    for r in results:
        for box, cls in zip(r.boxes.xyxy, r.boxes.cls):
            # COCO classes for vehicles: car=2, bus=5, truck=7, motorcycle=3, bicycle=1
            if int(cls) in [2, 3, 5, 7, 1]:
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(mask, (x1, y1), (x2, y2), 255, -1)
    return mask

In [24]:
def dual_plane_stabilization(frames_path, output_path, yolo_weights, plane1_ratio=0.7, feature_num=1000):
    print("[Init] Starting dual-plane stabilization with masking...")
    
    frame_files = sorted(glob(os.path.join(frames_path, "*.jpg")))
    if not frame_files:
        print("[Error] No JPG frames found!")
        return

    print(f"[Init] Found {len(frame_files)} frames")
    
    first_frame = cv2.imread(frame_files[0])
    h, w = first_frame.shape[:2]
    crop_margin = int(w * 0.05)
    output_res = (w-2*crop_margin, h-2*crop_margin)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, output_res)

    orb = cv2.ORB_create(feature_num)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    # Load YOLO with minimal verbosity
    print("[Init] Loading YOLO model...")
    yolo_model = YOLO(yolo_weights)
    yolo_model.verbose = False

    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    water_mask = create_water_mask(first_frame)
    vehicle_mask = create_vehicle_mask(first_frame, yolo_model)
    static_mask = cv2.bitwise_not(cv2.bitwise_or(water_mask, vehicle_mask))
    prev_kp, prev_des = orb.detectAndCompute(prev_gray, static_mask)

    start_time = time.time()
    print("[Status] Processing frames...")

    # Use tqdm for clean progress bar, no additional prints
    for i in tqdm(range(1, len(frame_files)), desc="Stabilizing", unit="frames"):
        curr_frame = cv2.imread(frame_files[i])
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        water_mask = create_water_mask(curr_frame)
        vehicle_mask = create_vehicle_mask(curr_frame, yolo_model)
        static_mask = cv2.bitwise_not(cv2.bitwise_or(water_mask, vehicle_mask))

        curr_kp, curr_des = orb.detectAndCompute(curr_gray, static_mask)
        if curr_kp is None or prev_kp is None or len(curr_kp) < 5 or len(prev_kp) < 5:
            prev_gray = curr_gray
            prev_kp, prev_des = curr_kp, curr_des
            continue

        matches = bf.match(prev_des, curr_des)
        matches = sorted(matches, key=lambda x: x.distance)[:int(len(matches)*plane1_ratio)]

        motion_vectors = [np.array(curr_kp[m.trainIdx].pt) - np.array(prev_kp[m.queryIdx].pt) for m in matches]
        if len(motion_vectors) < 2:
            prev_gray = curr_gray
            prev_kp, prev_des = curr_kp, curr_des
            continue

        # Suppress sklearn warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            kmeans = KMeans(n_clusters=2, random_state=0, n_init=10)
            labels = kmeans.fit_predict(np.array(motion_vectors).astype(np.float32))

        # Dual-plane homography
        H1, H2 = None, None
        plane1_pts = [(prev_kp[m.queryIdx].pt, curr_kp[m.trainIdx].pt) for m, l in zip(matches, labels) if l == 0]
        if len(plane1_pts) > 4:
            pts1, pts2 = zip(*plane1_pts)
            H1, _ = cv2.findHomography(np.array(pts1), np.array(pts2), cv2.RANSAC, 5.0)
        plane2_pts = [(prev_kp[m.queryIdx].pt, curr_kp[m.trainIdx].pt) for m, l in zip(matches, labels) if l == 1]
        if len(plane2_pts) > 4:
            pts1, pts2 = zip(*plane2_pts)
            H2, _ = cv2.findHomography(np.array(pts1), np.array(pts2), cv2.RANSAC, 5.0)

        stabilized = curr_frame.copy()
        if H1 is not None and len(plane1_pts) > 4:
            warped_plane1 = cv2.warpPerspective(curr_frame, H1, (w, h))
            mask = np.zeros_like(curr_gray)
            contour = cv2.convexHull(np.array([pt[1] for pt in plane1_pts], dtype=np.int32))
            cv2.drawContours(mask, [contour], -1, 255, -1)
            stabilized = cv2.bitwise_and(warped_plane1, warped_plane1, mask=mask) + \
                         cv2.bitwise_and(stabilized, stabilized, mask=~mask)
        if H2 is not None and len(plane2_pts) > 4:
            warped_plane2 = cv2.warpPerspective(curr_frame, H2, (w, h))
            mask = np.zeros_like(curr_gray)
            contour = cv2.convexHull(np.array([pt[1] for pt in plane2_pts], dtype=np.int32))
            cv2.drawContours(mask, [contour], -1, 255, -1)
            stabilized = cv2.bitwise_and(warped_plane2, warped_plane2, mask=mask) + \
                         cv2.bitwise_and(stabilized, stabilized, mask=~mask)

        stabilized = stabilized[crop_margin:h-crop_margin, crop_margin:w-crop_margin]
        out.write(stabilized)

        prev_gray = curr_gray
        prev_kp, prev_des = curr_kp, curr_des

    out.release()
    total_time = time.time() - start_time
    print(f"\n[Complete] Processed {len(frame_files)} frames in {total_time:.1f}s")
    print(f"[Complete] Average speed: {len(frame_files)/total_time:.1f} FPS")
    print(f"[Complete] Output saved: {output_path}")

# Usage
frames_path = r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\unzipped_video"
output_path = r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\yolo+dual+mask.mp4"
yolo_weights = "yolov8n.pt"
dual_plane_stabilization(frames_path, output_path, yolo_weights)

[Init] Starting dual-plane stabilization with masking...
[Init] Found 7169 frames
[Init] Loading YOLO model...

0: 384x640 (no detections), 93.2ms
Speed: 3.1ms preprocess, 93.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[Status] Processing frames...


Stabilizing:   0%|          | 0/7168 [00:00<?, ?frames/s]


0: 384x640 (no detections), 91.6ms
Speed: 2.6ms preprocess, 91.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


Stabilizing:   0%|          | 1/7168 [00:00<56:01,  2.13frames/s]


0: 384x640 (no detections), 118.6ms
Speed: 1.9ms preprocess, 118.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)


...


0: 384x640 (no detections), 114.9ms
Speed: 2.1ms preprocess, 114.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


Stabilizing: 100%|█████████▉| 7167/7168 [1:34:16<00:00,  1.65frames/s]


0: 384x640 (no detections), 112.6ms
Speed: 2.4ms preprocess, 112.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


Stabilizing: 100%|██████████| 7168/7168 [1:34:17<00:00,  1.27frames/s]


[Complete] Processed 7169 frames in 5657.3s
[Complete] Average speed: 1.3 FPS
[Complete] Output saved: C:\Users\simar\OneDrive\Desktop\Python_stabilization\yolo+dual+mask.mp4
